In [26]:
import numpy as np
import matplotlib.pyplot as plt
import napari
import tifffile as tiff
from aicsimageio import AICSImage

In [4]:
img = tiff.imread('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/lymphoma_CODEX/images/191_1_reg001_X01_Y01_Z01.tif')

In [7]:
img.shape

(52, 2, 3750, 4998)

In [10]:
img[:,0,:,:].shape

(52, 3750, 4998)

In [11]:
viewer = napari.Viewer()
viewer.add_image(img[:,0,:,:], name='img_0')
viewer.add_image(img[:,1,:,:], name='img_1')

<Image layer 'img_1' at 0x459695f30>

In [20]:
with tiff.TiffFile('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/lymphoma_CODEX/images/191_1_reg001_X01_Y01_Z01.tif') as tif:
    img = tif.asarray()
    md = tif.imagej_metadata
    pages = tif.pages

In [24]:
pages[0].description

'ImageJ=1.53p\nimages=104\nchannels=2\nslices=52\nhyperstack=true\nmode=grayscale\nloop=false\nmin=0.0\nmax=65535.0'